In [4]:
import tensorflow as tf
from tensorflow.keras import layers, applications, models
import numpy as np
from utilities.distiller import Distiller
from utilities.spots_10_loader import SPOT10Loader
import pandas as pd
import os

In [9]:
def create_teacher_model(model_name="MobileNet", input_shape=(32, 32, 3), num_classes=10):
    base_model = getattr(applications, model_name)(
        weights="imagenet",
        include_top=False,
        input_shape=input_shape
    )

    teacher = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes)
    ])

    return teacher

In [5]:
def create_student_model(input_shape=(32, 32, 3), num_classes=10):
    student = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),

        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),

        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(num_classes),
    ], name="student")

    return student


In [4]:
def load_data(dataset_dir="dataset", kind="train", input_shape=(32, 32, 3)):
    x_data, y_label = SPOT10Loader.get_data(dataset_dir=dataset_dir, kind=kind)

    x_data = x_data.astype('float32') / 255.0
    x_data = np.expand_dims(x_data, axis=-1)
    y_label = np.expand_dims(y_label, axis=-1)
    x_data = np.repeat(x_data, 3, axis=-1)

    return x_data, y_label

In [5]:
x_train, y_train = load_data(dataset_dir="dataset", kind="train", input_shape=(32, 32, 3))
x_test, y_test = load_data(dataset_dir="dataset", kind="test", input_shape=(32, 32, 3))

In [6]:
# "MobileNet", "ResNet50", "MobileNetV2", "DenseNet121", 
# "NASNetMobile", "MobileNetV3Small", "ResNet101", 
# MobileNetV3Large, "ResNet50V2", "ResNet101V2"
teacher_names=["ResNet50V2", "ResNet101V2"]

In [ ]:
student = create_student_model()

for teacher_name in teacher_names:
    print("Training benchmark model: ", teacher_name, "...")
    teacher = create_teacher_model(model_name=teacher_name)

    distiller = Distiller(student=student, teacher=teacher)
    distiller.compile(
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
        student_loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        distillation_loss_fn=tf.keras.losses.KLDivergence(),
        alpha=0.1,
        temperature=10,
    )
    
    # Define the ReduceLROnPlateau callback
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_sparse_categorical_accuracy',  # Monitor the student loss
        factor=0.5,              # Reduce learning rate by a factor of 0.5
        patience=5,              # Number of epochs with no improvement after which learning rate will be reduced
        min_lr=1e-6,             # Lower bound on the learning rate
        verbose=1                # Output reduction process
    )
    
    # Define the ModelCheckpoint callback
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=teacher_name + '_best_model',  # Path to save the model, SavedModel format by default
        monitor='val_sparse_categorical_accuracy',  # Monitor the validation accuracy
        save_best_only=True,  # Save only the best model
        mode='max',  # Mode for the monitored metric, 'max' for accuracy
        verbose=1,  # Output saving process
        save_format='tf'  # Save in TensorFlow SavedModel format
    )

    history = distiller.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test),  callbacks=[reduce_lr, checkpoint])


In [12]:
teacher_names=["MobileNet", "ResNet50", "MobileNetV2", 
               "DenseNet121", "NASNetMobile", "MobileNetV3Small", 
               "ResNet101",  "MobileNetV3Large", "ResNet50V2", "ResNet101V2"]

columns = ['model_name', 'accuracy']
df = pd.DataFrame(columns=columns)

for teacher_name in teacher_names:
    # Load the saved model
    saved_model_path = teacher_name+'_best_model'  # Path to the directory containing the saved model
    
    # Load the model
    model = tf.keras.models.load_model(saved_model_path)
    test_accuracy = model.evaluate(x_test, y_test)
    
    ## Save results to DataFrame
    new_data = pd.DataFrame({'model_name': [teacher_name], 'accuracy': [test_accuracy[1]]})
    df = pd.concat([df, new_data], ignore_index=True)



313/313 [==============================] - 2s 6ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 0.8029


In [16]:
# Create the destination folder for the benchmark results is not exist
save_directory = "benchmark_results"
os.makedirs(save_directory, exist_ok=True)

csv_file = save_directory + "/benchmark_accuracies.csv"
# Save DataFrame to CSV
df.to_csv(csv_file, index=False)
print(f"Test Accuracy results saved to {csv_file}")

Test Accuracy results saved to benchmark_results/benchmark_accuracies.csv


In [6]:
student = create_student_model()

In [8]:
student.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16, 16, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 8, 8, 64)          0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                       